In [ ]:
# !pip install requests
# !pip install Pillow
# !pip install deepface
# !pip install pandas

In [ ]:
import os
from deepface import DeepFace
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

# Read CSV file
df = pd.read_csv("input.csv")

dir_male = "images/male/"
dir_female = "images/female/"

# Create directories if they don't exist
os.makedirs(dir_male, exist_ok=True)
os.makedirs(dir_female, exist_ok=True)

for index, row in df.iterrows():
    file_name = row["Name"]
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    }
    response = requests.get(
        "https://www.thispersondoesnotexist.com", headers=headers)

    if response.status_code == 200 and "image" in response.headers["Content-Type"]:
        img = Image.open(BytesIO(response.content))

        # Crop the image
        width, height = img.size
        img = img.crop((20, 20, width - 20, height - 20))

        # Save image in the appropriate directory
        if row["Gender"] == "Male":
            full_file_name = dir_male + file_name + ".jpg"
        else:
            full_file_name = dir_female + file_name + ".jpg"

        img.save(full_file_name)

        print(
            f"Image {file_name} is predicted to be {row['Gender']} and saved to {full_file_name}"
        )
    else:
        print(
            f"Failed to download image {file_name}. Status code: {response.status_code}, Content-Type: {response.headers['Content-Type']}"
        )